# 如何传输自定义数据

<div class="警告提示">
<p class="admonition-title">先决条件</p>
<p>
本指南假设您熟悉以下内容：
<ul>
<li>
<a href="https://langchain-ai.github.io/langgraphjs/concepts/streaming/">
流媒体
</a>
</li>
<li>
<a href="https://js.langchain.com/docs/how_to/streaming#using-stream-events">
流事件 API
</a>
</li>
<li>
<a href="https://js.langchain.com/docs/concepts/chat_models">
聊天模型
</a>
</li>
<li>
<a href="https://js.langchain.com/docs/concepts/tools">
工具
</a>
</li>
</ul>
</p>
</div>

从节点内部进行流式传输的最常见用例是流式传输 LLM 令牌，但您可能还想流式传输自定义数据。

例如，如果您有一个长时间运行的工具调用，您可以在步骤之间分派自定义事件并使用这些自定义事件来监视进度。您还可以向应用程序的最终用户显示这些自定义事件，以向他们展示当前任务的进展情况。

您可以通过两种方式执行此操作：

* 使用图表的 `.stream` 方法和 `streamMode: "custom"`
* 使用 [`dispatchCustomEvents`](https://js.langchain.com/docs/how_to/callbacks_custom_events/) 和 `streamEvents` 发出自定义事件。

下面我们将了解如何使用这两个 API。

## 设置

首先，让我们安装所需的软件包：
```bash
npm install @langchain/langgraph @langchain/core
```

<div class="警告提示">
<p class="admonition-title">设置 <a href="https://smith.langchain.com">LangSmith</a> 进行 LangGraph 开发</p>
<p style="padding-top: 5px;">
注册 LangSmith 以快速发现问题并提高 LangGraph 项目的性能。LangSmith 允许您使用跟踪数据来调试、测试和监控使用 LangGraph 构建的 LLM 应用程序 - 在<a href="https://docs.smith.langchain.com">此处</a>了解有关如何开始的更多信息。
</p>
</div>

## 使用 .stream 流式传输自定义数据

<div class="警告提示">
<p class="admonition-title">兼容性</p>
<p>
此部分需要 <code>@langchain/langgraph>=0.2.20</code>。如需升级帮助，请参阅<a href="/langgraphjs/how-tos/manage-ecosystem-dependencies/">本指南</a>。
</p>
</div>

### 定义图表

In [1]:
import {
  StateGraph,
  MessagesAnnotation,
  LangGraphRunnableConfig,
} from "@langchain/langgraph";

const myNode = async (
  _state: typeof MessagesAnnotation.State,
  config: LangGraphRunnableConfig
) => {
  const chunks = [
    "Four",
    "score",
    "and",
    "seven",
    "years",
    "ago",
    "our",
    "fathers",
    "...",
  ];
  for (const chunk of chunks) {
    // 使用streamMode=custom写入要流式传输的块
    // 仅当传递的流模式之一为“自定义”时才填充。
    config.writer?.(chunk);
  }
  return {
    messages: [{
      role: "assistant",
      content: chunks.join(" "),
    }],
  };
};

const graph = new StateGraph(MessagesAnnotation)
  .addNode("model", myNode)
  .addEdge("__start__", "model")
  .compile();

### 流媒体内容

In [2]:
const inputs = [{
  role: "user",
  content: "What are you thinking about?",
}];

const stream = await graph.stream(
  { messages: inputs },
  { streamMode: "custom" }
);

for await (const chunk of stream) {
  console.log(chunk);
}

Four
score
and
seven
years
ago
our
fathers
...


您可能需要使用[多种流模式](https://langchain-ai.github.io/langgraphjs/how-tos/stream-multiple/)
希望访问自定义数据和状态更新。

In [3]:
const streamMultiple = await graph.stream(
  { messages: inputs },
  { streamMode: ["custom", "updates"] }
);

for await (const chunk of streamMultiple) {
  console.log(chunk);
}

[ 'custom', 'Four' ]
[ 'custom', 'score' ]
[ 'custom', 'and' ]
[ 'custom', 'seven' ]
[ 'custom', 'years' ]
[ 'custom', 'ago' ]
[ 'custom', 'our' ]
[ 'custom', 'fathers' ]
[ 'custom', '...' ]
[ 'updates', { model: { messages: [Array] } } ]


## 使用 .streamEvents 流式传输自定义数据

如果您已经在工作流程中使用图表的 `.streamEvents` 方法，您还可以通过使用 `dispatchCustomEvents` 发出自定义事件来流式传输自定义数据

### 定义图表

In [4]:
import { dispatchCustomEvent } from "@langchain/core/callbacks/dispatch";

const graphNode = async (_state: typeof MessagesAnnotation.State) => {
  const chunks = [
    "Four",
    "score",
    "and",
    "seven",
    "years",
    "ago",
    "our",
    "fathers",
    "...",
  ];
  for (const chunk of chunks) {
    await dispatchCustomEvent("my_custom_event", { chunk });
  }
  return {
    messages: [{
      role: "assistant",
      content: chunks.join(" "),
    }],
  };
};

const graphWithDispatch = new StateGraph(MessagesAnnotation)
  .addNode("model", graphNode)
  .addEdge("__start__", "model")
  .compile();

### 流媒体内容

In [5]:
const eventStream = await graphWithDispatch.streamEvents(
  {
    messages: [{
      role: "user",
      content: "What are you thinking about?",
    }]
  },
  {
    version: "v2",
  },
);

for await (const { event, name, data } of eventStream) {
  if (event === "on_custom_event" && name === "my_custom_event") {
    console.log(`${data.chunk}|`);
  }
}

Four|
score|
and|
seven|
years|
ago|
our|
fathers|
...|
